In [1]:
import sys
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/robustness_vs_counterfactuals")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/")
sys.path.append("/Users/skrishna/Documents/phd_codes/neurips_paper/robustness_vs_counterfactuals/Recourse_Methods/AR")

In [2]:

import numpy as np
import matplotlib.pyplot as plt
import matplotlib
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd

import ML_Models.data_loader as loader
# from Recourse_Methods.gradient_methods import SCFE
# from utils import get_recourses, get_performance_measures


%matplotlib inline
%config InlineBackend.figure_format = 'svg'

In [18]:
## Dataset Prep


from torchvision import  datasets, transforms
from torch.utils.data import DataLoader

dataset_name = "adult"
adult_dict = {
        "data_path": "../Data_Sets/Adult/",
        "filename_train": 'adult-train.csv',
        "filename_test": 'adult-test.csv',
        "label": 'income',
        "task": "classification",
        "lr": 1e-3,
        "d": 6,
        "H1": 25,
        "H2": 25,
#         "activFun": nn.Softplus(),
#         "n_starting_instances": n_starting_instances
    }

data_meta_dictionaries = {
        "adult": adult_dict
    }
data_meta_info = data_meta_dictionaries["adult"]


dataset_test = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                 filename=data_meta_info["filename_test"],
                                                 label=data_meta_info["label"])
        
dataset_train = loader.DataLoader_Tabular(path=data_meta_info["data_path"],
                                                  filename=data_meta_info["filename_train"],
                                                  label=data_meta_info["label"])


In [19]:
# Data loader

train_loader = DataLoader(dataset_train, batch_size = 32, shuffle=True)
test_loader = DataLoader(dataset_test, batch_size = 32, shuffle=False)

In [20]:
data = [i for i in train_loader]
num_input = len(data[0][0][0])

In [21]:
# Single pass over data 
# do a single pass over the data
def epoch(loader, model, opt=None):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        X = X.to(torch.float32)
        yp = model(X)[:,0]
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)

model = nn.Linear(num_input, 1)
opt = optim.SGD(model.parameters(), lr=1.)
print("Train Err", "Train Loss", "Test Err", "Test Loss", sep="\t")
for i in range(50):
    train_err, train_loss = epoch(train_loader, model, opt)
    test_err, test_loss = epoch(test_loader, model)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")
    
    

Train Err	Train Loss	Test Err	Test Loss
0.178981	0.375415	0.178662	0.369160
0.174724	0.365623	0.199226	0.409842
0.173425	0.361801	0.191155	0.385296
0.171711	0.360019	0.169154	0.356914
0.169279	0.357696	0.169928	0.350140
0.170025	0.357295	0.171365	0.359993
0.167841	0.355418	0.167496	0.349733
0.167372	0.354785	0.189386	0.393386
0.166045	0.354196	0.174682	0.368225
0.167095	0.353866	0.169044	0.353520
0.167012	0.353485	0.170923	0.356757
0.166211	0.353887	0.168270	0.357494
0.167482	0.353405	0.164069	0.347373
0.164607	0.352168	0.171144	0.363708
0.167676	0.352958	0.169596	0.355515
0.165630	0.352109	0.165174	0.349719
0.165437	0.353011	0.177004	0.381128
0.164524	0.351065	0.164290	0.350602
0.165575	0.351843	0.169596	0.357505
0.165685	0.352428	0.172250	0.355286
0.165105	0.351231	0.166059	0.351006
0.164303	0.351457	0.166390	0.344705
0.165851	0.352499	0.180100	0.366616
0.164331	0.350782	0.222996	0.445418
0.164303	0.351619	0.172250	0.358639
0.164856	0.351403	0.163184	0.344484
0.166653	0.352522	0.1731

In [34]:
# Save baseline model
torch.save(model, "models/{}_lr_model.pth".format(dataset_name))


In [24]:
# Specifying \delta
epsilon = 0.2
delta = epsilon * model.weight.detach().sign()


In [25]:
# Testing error on adversarial sample 
def epoch_adv(loader, model, delta):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        X = X.to(torch.float32)
#         tmp = ((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)
        yp = model(X-((2*y.float()-1)[:, None]*delta.squeeze(0)).squeeze(0)).squeeze(-1)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)
print(epoch_adv(test_loader, model, delta[None,None,:,:]))



(0.9540077390823659, 3.508329794748242)


In [26]:
# Robust model training 
def epoch_robust(loader, model, epsilon, opt=None):
    total_loss, total_err = 0.,0.
    for X,y,ind in loader:
        X = X.to(torch.float32)
        yp = model(X)[:,0] - epsilon*(2*y.float()-1)*model.weight.norm(1)
        loss = nn.BCEWithLogitsLoss()(yp, y.float())
        if opt:
            opt.zero_grad()
            loss.backward()
            opt.step()
        
        total_err += ((yp > 0) * (y==0) + (yp < 0) * (y==1)).sum().item()
        total_loss += loss.item() * X.shape[0]
    return total_err / len(loader.dataset), total_loss / len(loader.dataset)


model_robust = nn.Linear(num_input, 1)
opt = optim.SGD(model_robust.parameters(), lr=1e-1)
epsilon = 0.2
print("Rob. Train Err", "Rob. Train Loss", "Rob. Test Err", "Rob. Test Loss", sep="\t")
for i in range(50):
    train_err, train_loss = epoch_robust(train_loader, model_robust, epsilon, opt)
    test_err, test_loss = epoch_robust(test_loader, model_robust, epsilon)
    print(*("{:.6f}".format(i) for i in (train_err, train_loss, test_err, test_loss)), sep="\t")
    
    

Rob. Train Err	Rob. Train Loss	Rob. Test Err	Rob. Test Loss
0.253144	0.526429	0.247872	0.526535
0.247920	0.520663	0.247872	0.521500
0.248390	0.520488	0.247872	0.523071
0.248113	0.520579	0.247872	0.523466
0.248445	0.520769	0.247872	0.524076
0.248279	0.520683	0.247872	0.524673
0.248224	0.520786	0.247872	0.521711
0.247920	0.520824	0.247872	0.524016
0.248749	0.520484	0.247872	0.522677
0.248970	0.520508	0.247872	0.526301
0.248031	0.520764	0.247872	0.529104
0.247920	0.520816	0.247872	0.520596
0.248252	0.520464	0.247872	0.522629
0.248058	0.520676	0.247872	0.524411
0.248335	0.520616	0.247872	0.524170
0.248252	0.520818	0.247872	0.523619
0.248252	0.520887	0.247872	0.521508
0.248058	0.520574	0.247872	0.523541
0.248141	0.520867	0.247872	0.521935
0.247865	0.520706	0.247872	0.521994
0.248694	0.520396	0.247872	0.521338
0.247948	0.520485	0.247872	0.524958
0.249634	0.520760	0.247872	0.524254
0.248445	0.520885	0.247872	0.521430
0.248169	0.520671	0.247872	0.523915
0.248556	0.520542	0.247872	0.521234
0.24

In [31]:
# Save robust model 
torch.save(model_robust, "./models/{}_lr_model_robust.pth".format(dataset_name))
